### Getting Started with Keras Tuner

In [2]:
import tensorflow 
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.models import Sequential

import keras_tuner as kt

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("./diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Splitting Data

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape, y_train.shape

((576, 8), (576,))

#### Without Tuning

In [5]:
model = Sequential(
    [
        InputLayer(shape=(8, )),
        Dense(units=32, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=1, activation="sigmoid")
    ]
)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

I0000 00:00:1741078357.360347   14779 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2614 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [6]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100


I0000 00:00:1741078361.932508   15260 service.cc:148] XLA service 0x7816b4009230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741078361.932545   15260 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-03-04 14:22:41.951463: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741078362.089874   15260 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.5938 - loss: 5.8694

I0000 00:00:1741078362.701459   15260 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5352 - loss: 6.9728 - val_accuracy: 0.6510 - val_loss: 1.7089
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6675 - loss: 1.6365 - val_accuracy: 0.6510 - val_loss: 1.2811
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6349 - loss: 1.2632 - val_accuracy: 0.6771 - val_loss: 1.0124
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6517 - loss: 0.9637 - val_accuracy: 0.6510 - val_loss: 0.8608
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6545 - loss: 0.8295 - val_accuracy: 0.6250 - val_loss: 0.8095
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6871 - loss: 0.6759 - val_accuracy: 0.6823 - val_loss: 0.7726
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6333 - loss: 0.7021 - val_accuracy: 0.6562 - val_loss: 0.6687
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6419 - loss: 0.6855 - val_accuracy: 0.6042 - val_loss: 0.

#### With Tuning

First let us just try to tune one parameter at a time. 

We will tune
- optimizer
- number of nodes in a layer
- and finally number of layers (a bit complex)
- In the next section we will combine all of these and even more

In [8]:
# Tuning Optimizer

In [7]:
def build_model(hp):
    model = Sequential([
        InputLayer(shape=(8,)),
        Dense(32, activation="relu"),
        Dense(32, activation="relu"),
        Dense(1, activation="sigmoid")
    ])

    optimizer = hp.Choice("optimizer", ["adadelta", "adam", "rmsprop", "sgd"])

    model.compile(
        optimizer=optimizer, 
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )  

    return model


In [8]:
# Create a tuner object

tuner = kt.RandomSearch(                # can also give GridSearch()
    hypermodel=build_model,             # the build model function we made above
    objective="val_accuracy",           # the objective to improve upon
    max_trials=None,                    # number of parameter combinations to try - set to None to try out all combinations
                                        # (setting this less than total combinations will select only a subset of the search space)
    directory="tests",                  # the folder to store all the tests
    project_name="optimzer_tuning"      # the folder to store this tests results (must be unique for each tuning)
)


# search best parameters

tuner.search(
    X_train, y_train, 
    epochs=5,                               # train each combination for 5 epochs
    validation_data=(X_test, y_test))       # needed cuz objective is val_acccuracy (can also use validation split))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6614583134651184

Best val_accuracy So Far: 0.671875
Total elapsed time: 00h 00m 09s


In [9]:
# Tuning number of nodes in a layer

In [ ]:
def build_model(hp):

    model = Sequential(
        [
            InputLayer(shape=(8, )),
            Dense(units=hp.Integer(8, 32, 8), activation="relu"),
            Dense(units=hp.Integer(8, 32, 8), activation="relu"),
            Dense(units=1, activation="sigmoid")
        ]
    )

    model.compile(loss="binary_crossentropy", optimizer="adam")

    return model